In [1]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.utils.data as tud
import pdb
import random
import struct
import gzip

if torch.cuda.is_available(): #判断是否有gpu，就是看cuda能不能用
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
import warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode
landmarks_frame = pd.read_csv("../data/project2_train.csv")  #初步读取文件
dataset = landmarks_frame.values   
#把表格的数据直接放入变量中
#此时dataset的形状为[28706,2] 28706行，2列
#第一列是lable类型为int 第二列是data，是一整个字符串，里面包含图像像素值，用空格隔开，类型是string
data = dataset[:,1:]  #第二列数据取出来，此时data形状为[28706,1]
lable = dataset[:,:1] #第一列lable取出来 此时lable形状为[28706,1]

def to_int(x):   #此函数用于把string字符串转换成float的list
    return [float(a) for a in x[0].split()]  #split函数可以把string分割开

data = [to_int(x) for x in data]  #把string的data变成了形状为[28706,2304]
data = np.array(data)   #把list转换成numpy类型的矩阵，形状还是[28706,2304]为之后的操作做准备
datas = [[x.reshape(48,48)] for x in data]  
#把data中的数据变成[48*48]类型的数据，此时datas形状为[28709,1,48,48]
#这种数据形状是最终我们需要的形状
#含义为28709个数据，1维，大小为48*48
datas = np.array(datas) #datas的类型从list转换为numpy
lable = np.array(lable) #lable的类型从list转换为numpy
lable = lable.astype(np.int) #把lable里面数据强转为int
datas = datas.astype(np.float32) #把atas里面数据强转为float32

In [5]:
training_image = torch.from_numpy(datas) #把datas变成torch类型的矩阵(torch可以使用gpu)
training_lable = torch.from_numpy(lable) #同理
training_lable = training_lable.squeeze(1)
#这一句是把lable减少一维，本来training_lable的形状是[28709,1]
#运行完这一句之后就变成了[28709]

#最终完成的数据的初步处理
#training_image形状为[28709,1,48,48]
#training_lable形状为[28709] 
#都是torch类型的矩阵

torch.Size([28709, 1, 48, 48]) torch.Size([28709])
(28709, 1, 48, 48)


In [2]:
from __future__ import print_function, division
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.utils.data as tud
import pdb
import random
import struct
import gzip

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
import warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode
landmarks_frame = pd.read_csv("../data/project2_train.csv")
dataset = landmarks_frame.values
data = dataset[:,1:]
lable = dataset[:,:1]

In [7]:
print(dataset[0][0])

AttributeError: 'int' object has no attribute 'type'

In [3]:
training_image = training_image[:2000]
training_lable = training_lable[:2000]
print(training_image.shape)

torch.Size([2000, 1, 48, 48])


In [6]:
#接下来使用dataset将数据封装成batch(成组)
#dataset类有三个重要函数，
#__init__初始化 在创建此类的时候魂运行一次，用于将数据读入类中
#__len__测量大小，用于和下面那个函数配合使用
#__getitem__，此函数是dataset可以生成batch的关键函数，此函数的返回值代表取出一个数据。
#则此类会成组的调用__getitem__来生成数据，把每一个返回值拼装成成组的数据
#例如，batch_size设定为40，则此类就会成组的运行40次__getitem__来返回数据，每40次把数据合成一组
#一共运行__len__返回值除以40次，也就是生成__len__返回值除以40组的数据
class ExpressionClassificationDataset(tud.Dataset): #定义数据读取类，此类用于读取数据，最为关键的函数是getitem（）
    def __init__(self,training_image,training_lable, CEL): #此函数用于初始化
        super(ExpressionClassificationDataset, self).__init__()
        self.training_data = training_image.to(DEVICE)
        if(CEL == True): #CEL 代表CrossEntropyLoss 此loss方程需要需要的结果y和其他loss方程不一样（单维），所以需要另外处理
            self.training_lable = training_lable.to(DEVICE)
        else:
            self.training_lable = torch.zeros(len(training_lable),7,device = DEVICE)
            for i in range(len(training_lable)):
                self.training_lable[i][training_lable[i]] = 1
        
    def __len__(self): #此函数用于确定总数据大小，用于生成所有的bitch
        return len(self.training_data)
    def __getitem__(self, idx):
        
        return self.training_data[idx]/255.,self.training_lable[idx]; #/255是把数据的大小限制在0-1之间，能极大减少学习的时间
            
        
#此类的具体用下在下一运行格中

In [ ]:
#示例如下，读取training_image和training_lable
dataset = ExpressionClassificationDataset(training_image,training_lable,CEL = CEL)#用training_image和training_lable生成dataset类
train_loader = tud.DataLoader(dataset,batch_size = 40, shuffle = False)#把数据提取成组,shuffle的意思是是否随机抽取
#这一句所生成的train_loader的结构为[718,[40,1,48,48],[40]]
#调用方法是 for i,(x,y) in enumerate (train_loader): #每次取出的x形状为[40,1,48,48],y形状为[40]


In [55]:
#------------------------------用50行代码搭建ResNet-------------------------------------------
import torch as t
 
class ResidualBlock(nn.Module):
    #实现子module: Residual    Block
    def __init__(self,inchannel,outchannel,stride=1,shortcut=None):
        super(ResidualBlock,self).__init__()
        self.left=nn.Sequential(
            nn.Conv2d(inchannel,outchannel,3,stride,1,bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel,outchannel,3,1,1,bias=False),
            nn.BatchNorm2d(outchannel)
        )
        
        self.right=shortcut
        
    def forward(self,x):
        out=self.left(x)
        residual=x if self.right is None else self.right(x)
        out+=residual
        return F.relu(out)
    
    
class ResNet(nn.Module):
    #实现主module:ResNet34
    #ResNet34包含多个layer,每个layer又包含多个residual block
    #用子module实现residual block , 用 _make_layer 函数实现layer
    def __init__(self,num_classes=1000):
        super(ResNet,self).__init__()
        #重复的layer,分别有3,4,6,3个residual block
        self.BN = nn.BatchNorm2d(1)
        self.layer1=self._make_layer(1,64,3)
        self.layer2=self._make_layer(64,128,4,stride=2)
        self.layer3=self._make_layer(128,256,6,stride=2)
        self.layer4=self._make_layer(256,512,3,stride=2)
        
        #分类用的全连接
        self.fc=nn.Linear(512,num_classes)
        
    def _make_layer(self,inchannel,outchannel,block_num,stride=1):
        #构建layer,包含多个residual block
        shortcut=nn.Sequential(
            nn.Conv2d(inchannel,outchannel,1,stride,bias=False),
            nn.BatchNorm2d(outchannel))
 
        layers=[ ]
        layers.append(ResidualBlock(inchannel,outchannel,stride,shortcut))
        
        for i in range(1,block_num):
            layers.append(ResidualBlock(outchannel,outchannel))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        x = self.BN(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=F.avg_pool2d(x,6)
        x=x.view(x.size(0),-1)
        return self.fc(x)

In [52]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,20,3) 
#         48-3+1 = 46
#         23-3+1 = 21
#         21-2+1 = 20
        #pooling
        self.conv2 = torch.nn.Conv2d(20,20,3) 
        self.conv3 = torch.nn.Conv2d(20,20,2) 
        self.linear1 = torch.nn.Linear(20*10*10,1000)
        self.linear4 = torch.nn.Linear(1000,500)
        self.linear5 = torch.nn.Linear(500,200)
        self.linear6 = torch.nn.Linear(200,100)
        self.linear7 = torch.nn.Linear(100,7)
    def forward(self,x):
        x = F.relu(self.conv1(x))
#         print(x[0][1])
#         pdb.set_trace()
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1,20*10*10)
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = F.relu(self.linear6(x))
        x = F.relu(self.linear7(x))
#         print(F.softmax(x, dim = 1)) #调试语句
#         pdb.set_trace()
        return x

In [7]:
# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)

# for i,(x,y) in enumerate (train_loader):
#     if i<1:
#         print(x)
#         print(y)
#         break

In [8]:
def test(model,test_number,test_loader): #测试函数
    total_correct = 0.0
    for i, (x,y) in enumerate(test_loader):
        y_pred = model.forward(x)  #前项传播
        correct = sum(a == torch.argmax(b) for a, b in zip(y,y_pred))
        total_correct = total_correct+correct
#             pdb.set_trace()
        
    return total_correct/test_number

In [57]:
CEL = True
eta = 0.01
batch_size = 10
NUM_EPOCHS = 30
n_train = len(training_image)
split = n_train // 10   #整数除法，要把数据分成9:1的训练集和测试集
test_number = split  #测试集总数
indices = list(range(n_train))
train_sampler = tud.sampler.SubsetRandomSampler(indices[split:])  #设置采样器
test_sampler = tud.sampler.SubsetRandomSampler(indices[:split])
dataset = ExpressionClassificationDataset(training_image,training_lable,CEL = CEL)
train_loader = tud.DataLoader(dataset, sampler=train_sampler,batch_size = batch_size, shuffle = False)#利用采样的方法进行提取
test_loader = tud.DataLoader(dataset, sampler=test_sampler, batch_size = batch_size, shuffle = False)

model = Net()
loss_fn = nn.CrossEntropyLoss() #定义loss_fn，loss方程
optimizer = torch.optim.SGD(model.parameters(), lr = eta)#定义参数更新方式
for e in range(NUM_EPOCHS): #每一个epochs
    for i,(x,y) in enumerate (train_loader):
#从dataloader中取出bitch数据进行训练
#               a = x[0].reshape(48,48) #显示图片
#               fig = plt.figure()
#               plotwindow = fig.add_subplot(111)
#               plt.imshow(a , cmap='gray')
#               plt.show()
#               print(y[0])
        optimizer.zero_grad() #参数更新器重置为0
        y_pred = model.forward(x)  #前项传播 
#         if(i % 5 ==0):
#             print(y_pred) 
        loss = loss_fn(y_pred,y.long()) #计算loss
        loss.backward()   # 反向传播
        optimizer.step() #参数更新
#     print(y_pred)
    print(e,":轮，loss = ",loss.item())
    print("准确率为：",test(model,test_number,test_loader));





0 :轮，loss =  1.9381694793701172
准确率为： tensor(0.2350)
1 :轮，loss =  1.8617029190063477
准确率为： tensor(0.2350)
2 :轮，loss =  1.8915783166885376
准确率为： tensor(0.2350)
3 :轮，loss =  1.9033114910125732
准确率为： tensor(0.2350)
4 :轮，loss =  1.9873697757720947
准确率为： tensor(0.2350)
5 :轮，loss =  1.8686262369155884
准确率为： tensor(0.2350)
6 :轮，loss =  1.953998327255249
准确率为： tensor(0.2350)
7 :轮，loss =  1.8475284576416016
准确率为： tensor(0.2350)
8 :轮，loss =  1.9234898090362549
准确率为： tensor(0.2350)
9 :轮，loss =  1.8503059148788452
准确率为： tensor(0.2350)
10 :轮，loss =  1.9596607685089111
准确率为： tensor(0.2350)
11 :轮，loss =  1.8871047496795654
准确率为： tensor(0.2350)
12 :轮，loss =  1.9606313705444336
准确率为： tensor(0.2350)
13 :轮，loss =  1.812024712562561
准确率为： tensor(0.2350)
14 :轮，loss =  1.8859474658966064
准确率为： tensor(0.2350)
15 :轮，loss =  1.8862431049346924
准确率为： tensor(0.2350)
16 :轮，loss =  1.8120044469833374
准确率为： tensor(0.2350)
17 :轮，loss =  1.9231780767440796
准确率为： tensor(0.2350)
18 :轮，loss =  1.925299882888794
准确率为： te

In [11]:
class ExpressionClassification(object):
    def __init__(self,training_image,training_lable,eta,batch_size,shuffle,CEL): #CEL 代表使用交叉熵计算loss
        self.CEL = CEL
        self.dataset = ExpressionClassificationDataset(training_image,training_lable,CEL = self.CEL)
        self.split_9_1(len(training_image),batch_size) #把数据分为训练集和测试集  详细过程见下面的那个split函数
        self.dataloader = tud.DataLoader(self.dataset, batch_size=batch_size, shuffle=shuffle)
        if torch.cuda.is_available():
            self.model = ResNet(7).cuda()  #定义模型类
        else:
            self.model = ResNet(7)
        if (CEL == True):
            self.loss_fn = torch.nn.CrossEntropyLoss() #定义loss_fn，loss方程
        else:
            self.loss_fn = torch.nn.MSELoss() #定义loss_fn，loss方程
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr = eta)#定义参数更新方式
    def train(self): #开始训练
        for e in range(NUM_EPOCHS): #每一个epochs
            for i,(x,y) in enumerate (self.train_loader): #从dataloader中取出bitch数据进行训练
#                 a = x[0].reshape(28,28) #显示图片
#                 fig = plt.figure()
#                 plotwindow = fig.add_subplot(111)
#                 plt.imshow(a , cmap='gray')
#                 plt.show()
#                 print(y[0])
                y_pred = self.model.forward(x)  #前项传播
                
                
                if self.CEL == True:
                    self.loss = self.loss_fn(y_pred,y.long()) #计算loss
                else:
                    self.loss = self.loss_fn(y_pred,y) #计算loss
                if(i % 5 ==0):
                    print(y_pred)
#                 print(self.loss)
                print(self.loss)
                self.optimizer.zero_grad() #参数更新器重置为0
                self.loss.backward()   # 反向传播
                self.optimizer.step() #参数更新
                
            print(e,":轮，loss = ",self.loss.item())
            print("准确率为：",self.test());
    def test(self): #测试函数
        total_correct = 0.0
        for i, (x,y) in enumerate(self.test_loader):
            y_pred = self.model.forward(x)  #前项传播
            if self.CEL == True:  #对于不同loss方程的处理
                correct = self.correct_count_CEL(y,y_pred)
            else:
                correct = self.correct_count_MSE(y,y_pred)

            total_correct = total_correct+correct
#             pdb.set_trace()
        
        return total_correct/self.test_number
    def correct_count_MSE(self,y,y_pred): #MES正确预测数量函数
        return sum(a[torch.argmax(b)] == 1 for a, b in zip(y,y_pred))
    def correct_count_CEL(self,y,y_pred): #CrossEntropyLoss正确预测数量函数
        return sum(a == torch.argmax(b) for a, b in zip(y,y_pred))
    def split_9_1(self,n_train,batch_size):
        split = n_train // 10   #整数除法，要把数据分成9:1的训练集和测试集
        self.test_number = split  #测试集总数
        indices = list(range(n_train))
        train_sampler = tud.sampler.SubsetRandomSampler(indices[split:])  #设置采样器
        test_sampler = tud.sampler.SubsetRandomSampler(indices[:split])
        self.train_loader = tud.DataLoader(self.dataset, sampler=train_sampler,batch_size = batch_size, shuffle=False)#利用采样的方法进行提取
        self.test_loader = tud.DataLoader(self.dataset, sampler=test_sampler, batch_size = batch_size, shuffle=False)

In [ ]:
NUM_EPOCHS = 30
EC = ExpressionClassification(training_image,training_lable,eta = 0.01,batch_size = 64,shuffle = False,CEL = True)
EC.train()

tensor([[-0.1072,  0.1080,  0.3656, -0.5559,  0.8535, -0.1481,  0.5055],
        [-0.3731, -0.0215,  0.4637, -0.4113,  0.7316, -0.2247,  0.7401],
        [-0.0631, -0.0797,  0.5682, -0.3058,  0.7862, -0.1177,  0.6846],
        [-0.3286,  0.3317,  0.4767, -0.1870,  0.8966,  0.2834,  0.4463],
        [-0.2926,  0.0886,  0.4207, -0.1479,  0.7421, -0.1126,  0.6714],
        [-0.2461,  0.0400,  0.2240, -0.2032,  0.8572,  0.1783,  0.6081],
        [-0.4260,  0.3748,  0.3977, -0.0313,  0.7176,  0.2455,  0.6103],
        [-0.1150,  0.1483,  0.2892, -0.2626,  0.8061,  0.1230,  0.6754],
        [-0.2013,  0.1650,  0.4929, -0.1216,  0.6711,  0.1234,  0.6325],
        [-0.0915,  0.2254,  0.5042, -0.0584,  0.5878,  0.2417,  0.7225],
        [-0.2995, -0.1190,  0.4811, -0.2755,  0.8314, -0.1850,  0.6385],
        [-0.1505,  0.3400,  0.2787, -0.0673,  0.6915,  0.3559,  0.6501],
        [-0.2859,  0.3486,  0.1959,  0.0484,  0.8756,  0.1914,  0.6920],
        [-0.3199,  0.2640,  0.3216, -0.1047,  0.769

tensor(1.8525, grad_fn=<NllLossBackward>)
tensor(2.0398, grad_fn=<NllLossBackward>)
tensor(1.8246, grad_fn=<NllLossBackward>)
tensor(1.8266, grad_fn=<NllLossBackward>)


(28709, 1)


In [2]:
import pdb
EPOCH = 30
BITH_SIZE = 50
learning_rate = 0.001
n = len(training_data)
n_test = len(test_data)
test_datas = torch.zeros((n_test,1,48,48),device = DEVICE)
test_lables = torch.zeros((n_test,7),device = DEVICE)
test_datas = test_datas[:1000]
test_lables = test_lables[:1000]
test_data = torch.from_numpy(test_data)
test_lable = torch.from_numpy(test_lable)
for x in range(1000) :
    test_datas[x] = test_data[x]
    test_lables[x] = test_lable[x]
training_data = torch.from_numpy(training_data)
training_lable = torch.from_numpy(training_lable)
training_datas = torch.zeros((int(n/BITH_SIZE),BITH_SIZE,1,48,48),device = DEVICE)
training_lables = torch.zeros((int(n/BITH_SIZE),BITH_SIZE,7),device = DEVICE)
for x,y in zip(range(0,n,BITH_SIZE),range(int(n/BITH_SIZE))):
    training_datas[y] = training_data[x:x+BITH_SIZE]
    training_lables[y] = training_lable[x:x+BITH_SIZE]
# training_datas = training_datas[:1000]  #测试用
# training_lables = training_lables[:1000]

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,20,3) 
#         48-3+1 = 46
#         23-3+1 = 21
#         21-2+1 = 20
        #pooling
        self.conv2 = torch.nn.Conv2d(20,20,3) 
        self.conv3 = torch.nn.Conv2d(20,20,2) 
        self.linear1 = torch.nn.Linear(20*10*10,3000)
        self.linear2 = torch.nn.Linear(3000,2000)
        self.linear3 = torch.nn.Linear(2000,1000)
        self.linear4 = torch.nn.Linear(1000,500)
        self.linear5 = torch.nn.Linear(500,200)
        self.linear6 = torch.nn.Linear(200,7)
    def forward(self,x):
        x = F.relu(self.conv1(x))
#         print(x[0][1])
#         pdb.set_trace()
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1,20*10*10)
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = self.linear6(x)
#         print(F.softmax(x, dim = 1)) #调试语句
#         pdb.set_trace()
        return F.log_softmax(x, dim = 1)
model = Net()
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
training_datas = training_datas[:1000]
training_lables = training_lables[:1000]
for n in range(EPOCH):
#     i = 0
    for x,y in zip(training_datas,training_lables):

        y_pred = model.forward(x)
#         print(y.shape)
#         print(y_pred)
#         pdb.set_trace()
    #计算loss
#         i = i+1
#         print(i)
        loss = loss_fn(y_pred, y.long())
        print(loss)
        optimizer.zero_grad()
#         print(y)
#         pdb.set_trace()
        loss.backward()
    
        #update model parameters
        optimizer.step()
    print(loss)
    print(n,"测试结果:",test())
def test():
    xxx = torch.zeros(1000,1)
    a = model.forward(test_datas)
    for k,i in zip(a,range(1000)):
        xxx[i] = torch.argmax(k)
    
    return sum(y[int(x)] == 1 for x, y in zip(xxx,test_lables))




RuntimeError: 1D target tensor expected, multi-target not supported

In [14]:
import pdb
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1,20,3) 
#         48-3+1 = 46
#         23-3+1 = 21
#         21-2+1 = 20
        #pooling
        self.conv2 = torch.nn.Conv2d(20,20,3) 
        self.conv3 = torch.nn.Conv2d(20,20,2) 
        self.linear1 = torch.nn.Linear(20*10*10,3000)
        self.linear2 = torch.nn.Linear(3000,2000)
        self.linear3 = torch.nn.Linear(2000,1000)
        self.linear4 = torch.nn.Linear(1000,500)
        self.linear5 = torch.nn.Linear(500,200)
        self.linear6 = torch.nn.Linear(200,7)
    def forward(self,x):
        x = F.relu(self.conv1(x))
#         print(x[0][1])
#         pdb.set_trace()
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1,20*10*10)
        
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = F.relu(self.linear3(x))
        x = F.relu(self.linear4(x))
        x = F.relu(self.linear5(x))
        x = self.linear6(x)
#         print(F.softmax(x, dim = 1)) #调试语句
#         pdb.set_trace()
        return F.log_softmax(x, dim = 1)
        
class FaceNet(object):
    def __init__(self,training_data,training_lable,epoch,mini,eta,test_data = None, test_lable = None):
        n = len(training_data)
        n_test = len(test_data)
        self.test_datas = torch.zeros((n_test,1,48,48),device = DEVICE)
        self.test_lables = torch.zeros((n_test,7),device = DEVICE)
        self.test_datas = self.test_datas[:1000]
        self.test_lables = self.test_lables[:1000]
        test_data = torch.from_numpy(test_data)
        test_lable = torch.from_numpy(test_lable)
        for x in range(1000) :
            self.test_datas[x] = test_data[x]
            self.test_lables[x] = test_lable[x]
        
        print(n,mini)
        training_lable = training_lable[:20000]
        training_lable = training_lable.reshape(1,20000)
        training_lable = training_lable.astype(float)
        training_data = torch.from_numpy(training_data)
        training_lable = torch.from_numpy(training_lable)

        self.training_datas = torch.zeros((int(n/mini),mini,1,48,48),device = DEVICE)
        self.training_lables = torch.zeros((int(n/mini),mini),device = DEVICE)
        for x,y in zip(range(0,n,mini),range(int(n/mini))):
            self.training_datas[y] = training_data[x:x+mini]
            self.training_lables[y] = training_lable[0][x:x+mini]
        self.training_datas = self.training_datas[:100]  #测试用
        self.training_lables = self.training_lables[:100]
        self.epoch = epoch
        self.model = Net()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.learning_rate = eta
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = eta)
        
    def train(self):
        
        for n in range(self.epoch):
            #前项传播
            self.optimizer.zero_grad()
            for x,y in zip(self.training_datas,self.training_lables):
                y_pred = self.model.forward(x)
#                 print(y_pred)
#                 pdb.set_trace()
                #计算loss
                loss = self.loss_fn(y_pred,y.long())
                
                print(loss)
                
                loss.backward()
            
                #update model parameters
                self.optimizer.step()
            print(n,"测试结果:",self.test())
    def test(self):
        xxx = torch.zeros(1000,1)
        a = self.model.forward(test_datas)
        for k,i in zip(a,range(1000)):
            xxx[i] = torch.argmax(k)
            
        return sum(y[int(x)] == 1 for x, y in zip(xxx,self.test_lables))
    
    
    
    
    
            

In [15]:
net = FaceNet(training_data,training_lable,10,10,0.001,test_data, test_lable)
net.train()

20000 10
tensor(1.9297, grad_fn=<NllLossBackward>)
tensor(5.8809, grad_fn=<NllLossBackward>)
tensor(2.3519, grad_fn=<NllLossBackward>)
tensor(1.7551, grad_fn=<NllLossBackward>)
tensor(2.1638, grad_fn=<NllLossBackward>)
tensor(2.0111, grad_fn=<NllLossBackward>)
tensor(1.8681, grad_fn=<NllLossBackward>)
tensor(1.9684, grad_fn=<NllLossBackward>)
tensor(1.9105, grad_fn=<NllLossBackward>)
tensor(1.8171, grad_fn=<NllLossBackward>)
tensor(1.5498, grad_fn=<NllLossBackward>)
tensor(1.7739, grad_fn=<NllLossBackward>)
tensor(2.0472, grad_fn=<NllLossBackward>)
tensor(2.0133, grad_fn=<NllLossBackward>)
tensor(1.8597, grad_fn=<NllLossBackward>)
tensor(2.2332, grad_fn=<NllLossBackward>)
tensor(2.2678, grad_fn=<NllLossBackward>)
tensor(2.4702, grad_fn=<NllLossBackward>)
tensor(1.8423, grad_fn=<NllLossBackward>)
tensor(1.7323, grad_fn=<NllLossBackward>)
tensor(2.1067, grad_fn=<NllLossBackward>)
tensor(1.5934, grad_fn=<NllLossBackward>)
tensor(4.3727, grad_fn=<NllLossBackward>)
tensor(2.9540, grad_fn=<N

tensor(1.7823, grad_fn=<NllLossBackward>)
tensor(1.9428, grad_fn=<NllLossBackward>)
tensor(1.7483, grad_fn=<NllLossBackward>)
tensor(1.9404, grad_fn=<NllLossBackward>)
1 测试结果: tensor(265)
tensor(1.6990, grad_fn=<NllLossBackward>)
tensor(1.9423, grad_fn=<NllLossBackward>)
tensor(1.7143, grad_fn=<NllLossBackward>)
tensor(1.8754, grad_fn=<NllLossBackward>)
tensor(1.7097, grad_fn=<NllLossBackward>)
tensor(2.0223, grad_fn=<NllLossBackward>)
tensor(1.8432, grad_fn=<NllLossBackward>)
tensor(1.8241, grad_fn=<NllLossBackward>)
tensor(1.9823, grad_fn=<NllLossBackward>)
tensor(1.6555, grad_fn=<NllLossBackward>)
tensor(1.6941, grad_fn=<NllLossBackward>)
tensor(1.7470, grad_fn=<NllLossBackward>)
tensor(1.6712, grad_fn=<NllLossBackward>)
tensor(1.8051, grad_fn=<NllLossBackward>)
tensor(1.7245, grad_fn=<NllLossBackward>)
tensor(1.7803, grad_fn=<NllLossBackward>)
tensor(1.8081, grad_fn=<NllLossBackward>)
tensor(1.9366, grad_fn=<NllLossBackward>)
tensor(1.6902, grad_fn=<NllLossBackward>)
tensor(1.9286,

KeyboardInterrupt: 

In [19]:
def to_int(x):
    return [float(a) for a in x[0].split()]

data = [to_int(x) for x in data]
data = np.array(data)
lable = [x[0] for x in lable]
lable = np.array(lable)
data = [[x.reshape(48,48)] for x in data]
data = np.array(datas)
datas = torch.from_numpy(datas)
lables = torch.from_numpy(lable)

In [ ]:
datas = datas.double()
datas = datas[:20000]
lables = lables[:20000]

import pdb

for mini, x, y in datas_0:
    print(mini)
    print(x.shape)
    print(x)
    print(y.shape)
    print(y)
    pdb.set_trace()


0
torch.Size([10, 1, 48, 48])
tensor([[[[ 70.,  80.,  82.,  ...,  52.,  43.,  41.],
          [ 65.,  61.,  58.,  ...,  56.,  52.,  44.],
          [ 50.,  43.,  54.,  ...,  49.,  56.,  47.],
          ...,
          [ 91.,  65.,  42.,  ...,  72.,  56.,  43.],
          [ 77.,  82.,  79.,  ..., 105.,  70.,  46.],
          [ 77.,  72.,  84.,  ..., 106., 109.,  82.]]],


        [[[151., 150., 147.,  ..., 129., 140., 120.],
          [151., 149., 149.,  ..., 122., 141., 137.],
          [151., 151., 156.,  ..., 109., 123., 146.],
          ...,
          [188., 188., 121.,  ..., 185., 185., 186.],
          [188., 187., 196.,  ..., 186., 182., 187.],
          [186., 184., 185.,  ..., 193., 183., 184.]]],


        [[[231., 212., 156.,  ...,  44.,  27.,  16.],
          [229., 175., 148.,  ...,  27.,  35.,  27.],
          [214., 156., 157.,  ...,  28.,  22.,  28.],
          ...,
          [241., 245., 250.,  ...,  57., 101., 146.],
          [246., 250., 252.,  ...,  78., 105., 162.],

In [ ]:
def to_int(x):
    return [float(a) for a in x[0].split()]

data = [to_int(x) for x in data]
data = np.array(data)
lable = [x[0] for x in lable]
lable = np.array(lable)
datas = [[x.reshape(48,48)] for x in data]
datas = np.array(datas)
datas = torch.from_numpy(datas)
lables = torch.from_numpy(lable)
datas = datas[:20000]
lables = lables[:20000]
datas_0 = [
[n,datas[k:k+10],lables[k:k+10]]
for n,k,l in zip(range(2000),range(0, n, 10),range(0, n, 10))]